# L4: Optimize DSPy Agent with DSPy Optimizer

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

import os

os.environ["OPENAI_API_KEY"] = get_openai_api_key()

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.</p>

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

In [2]:
import mlflow

In [3]:
from helper import get_mlflow_tracking_uri

mlflow_tracking_uri = get_mlflow_tracking_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [4]:
mlflow.set_experiment("dspy_course_4")

2025/10/29 10:26:38 INFO mlflow.tracking.fluent: Experiment with name 'dspy_course_4' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/714844631808117306', creation_time=1761733598258, experiment_id='714844631808117306', last_update_time=1761733598258, lifecycle_stage='active', name='dspy_course_4', tags={}>

In [5]:
mlflow.dspy.autolog(log_evals=True, log_compiles=True, log_traces_from_compile=True)

In [6]:
import dspy

dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Build a RAG Agent

In [7]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

In [8]:
import json

# Load trainset
trainset = []
with open("trainset.jsonl", "r") as f:
    for line in f:
        trainset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

# Load valset
valset = []
with open("valset.jsonl", "r") as f:
    for line in f:
        valset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

In [9]:
# Overview of the dataset.
print(trainset[0])

Example({'question': 'Are Smyrnium and Nymania both types of plant?', 'answer': 'yes'}) (input_keys={'question'})


In [10]:
tp = dspy.MIPROv2(
    metric=dspy.evaluate.answer_exact_match,
    auto="light",
    num_threads=16
)

In [11]:
dspy.cache.load_memory_cache("./memory_cache.pkl")

In [12]:
optimized_react = tp.compile(
    react,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,
)

2025/10/29 10:26:41 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'abda392434b841e1ba73606aeb7c5de9', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/10/29 10:26:41 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/10/29 10:26:41 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/10/29 10:26:41 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/10/29 10:26:41 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 18%|█▊        | 18/100 [00:01<00:05, 14.00it/s]

Bootstrapped 4 full traces after 18 examples for up to 1 rounds, amounting to 18 attempts.


Bootstrapping set 4/6


...

2025/10/29 10:27:33 INFO dspy.evaluate.evaluate: Average Metric: 54 / 100 (54.0%)


2025/10/29 10:27:33 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 54.0
2025/10/29 10:27:33 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [31.0, 50.0, 49.0, 49.0, 54.0]
2025/10/29 10:27:33 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 54.0
2025/10/29 10:27:33 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/10/29 10:27:33 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/10/29 10:27:33 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 54.0!


🏃 View run eval_full_4 at: https://s172-29-112-60p8080.lab-aws-production.deeplearning.ai//#/experiments/714844631808117306/runs/360a759d35914c37b473aebff49f3d14
🧪 View experiment at: https://s172-29-112-60p8080.lab-aws-production.deeplearning.ai//#/experiments/714844631808117306


🏃 View run selective-goose-236 at: https://s172-29-112-60p8080.lab-aws-production.deeplearning.ai//#/experiments/714844631808117306/runs/abda392434b841e1ba73606aeb7c5de9
🧪 View experiment at: https://s172-29-112-60p8080.lab-aws-production.deeplearning.ai//#/experiments/714844631808117306


[Trace(request_id=6883e675dca94958869449ea216938b3), Trace(request_id=7120b313faa1448ba53b89f86be23d90), Trace(request_id=3a71032b49e24b93a48373f2cd9ae92d), Trace(request_id=ee850b086fa64828b0b415c68533703f), Trace(request_id=e8b3db787bb74cdc8dc14875e5eed431), Trace(request_id=0e6c9d88956a48fa92e15bf13207e819), Trace(request_id=0e011a8227e24d4ea6ffd47e6fabb579), Trace(request_id=237e014296854cec90e8cddd9c7174ed), Trace(request_id=476db09cc14f4a19905c47f0a7eff239), Trace(request_id=6a2055593d034572852dd699e5151e63)]

In [13]:
optimized_react.react.signature

StringSignature(question, trajectory -> next_thought, next_tool_name, next_tool_args
    instructions="Given the fields `question`, produce the fields `answer`.\n\nYou are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\nYour goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n\nTo do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\nAfter each tool call, you receive a resulting observation, which gets appended to your trajectory.\n\nWhen writing next_thought, you may reason about the current situation and plan for future steps.\nWhen selecting the next_tool_name and its next_tool_args, the tool must be one of:\n\n(1) search_wikipedia. It takes arguments {'query': {'type': 'string'}} in JSON format.\n(2) finish, whose description is <desc>Marks the task as complete. That is, signals

In [14]:
optimized_react.react.demos

[Example({'augmented': True, 'question': 'That Darn Cat! and Never a Dull Moment were both produced by what studio?', 'trajectory': '[[ ## thought_0 ## ]]\nI need to find out which studio produced both "That Darn Cat!" and "Never a Dull Moment." This information is likely available on Wikipedia, so I will search for it there.\n\n[[ ## tool_name_0 ## ]]\nsearch_wikipedia\n\n[[ ## tool_args_0 ## ]]\n{"query": "That Darn Cat! and Never a Dull Moment studio production"}\n\n[[ ## observation_0 ## ]]\n[1] «That Darn Cat! | That Darn Cat! is a 1965 American Walt Disney Productions thriller comedy film starring Hayley Mills (in her last of the six films she made for the Walt Disney Studios) and Dean Jones (starring in his first film for Disney) in a story about bank robbers, a kidnapping and a mischievous cat. The film was based on the 1963 novel "Undercover Cat" by Gordon and Mildred Gordon and was directed by Robert Stevenson. The title song was written by the Sherman Brothers and sung by Bo

In [15]:
evaluator = dspy.Evaluate(
    metric=dspy.evaluate.answer_exact_match,
    devset=valset,
    display_table=True,
    display_progress=True,
    num_threads=24,
)

In [16]:
original_score = evaluator(react)
print(f"Original score: {original_score}")

Average Metric: 31.00 / 100 (31.0%): 100%|██████████| 100/100 [00:03<00:00, 30.17it/s]

2025/10/29 10:27:36 INFO dspy.evaluate.evaluate: Average Metric: 31 / 100 (31.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...","Steve McQueen, known as ""the king of cool,"" starred in the movie ""...","The movie is ""The Great Escape.""",
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,"{'thought_0': 'I need to determine which individual, Robert Kardas...",Robert Kardashian's family is well-known for their reality TV show...,Robert Kardashian's family had their own reality TV show.,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in the film ""Shadows ...","I searched for information about the cast of the 1986 film ""Shadow...",There is no information available about a Russian ballerina in the...,
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,"{'thought_0': ""I need to find out who appointed Amashsai and the m...",Nehemiah appointed Amashsai to work at the temple in Jerusalem. Th...,"The meaning of the name of the man who appointed Amashsai, Nehemia...",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'I need to find out what additional requirements or ...,To gain access to 173 countries and territories with an Austrian p...,"In addition to the Austrian passport, travelers may need to obtain...",
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the name of the American actress...,The American actress and singer-songwriter known for her role as P...,2007,
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'I need to identify the animated creatures that were...,The animated creatures that are the title characters of the film b...,The animated creatures that are the title characters of the film b...,
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,{'thought_0': 'I need to verify the nationalities and contribution...,Both Dorothy Arzner and Richard Wallace were confirmed to be Ameri...,"No, neither Dorothy Arzner nor Richard Wallace were French film di...",


🏃 View run sneaky-skink-824 at: https://s172-29-112-60p8080.lab-aws-production.deeplearning.ai//#/experiments/714844631808117306/runs/10bef02e70f945d8ae901bbc832769ef
🧪 View experiment at: https://s172-29-112-60p8080.lab-aws-production.deeplearning.ai//#/experiments/714844631808117306
Original score: 31.0


[Trace(request_id=a72edf271b46403a9f24794b7cf5528d), Trace(request_id=4c844e5119b74d0f9cd27ec18555bc8d), Trace(request_id=95d1e11f3d66451398e4f9ebc90f107d), Trace(request_id=7f817fe2e0e14bb5adc36afc38172bee), Trace(request_id=190ffc012ac94f5b9e53bf671c4efc42), Trace(request_id=8c53141961d6469e8c2a432a28af57cc), Trace(request_id=34b05696c4334e7f8821e1600d0b8b6d), Trace(request_id=380d31db92344f3aafb7314b82cf314c), Trace(request_id=9466f4ce625a4765bcf30cfb9f7bf04e), Trace(request_id=b8e47bf8976f4449b3326f6604eeb5c0)]

In [17]:
optimized_score = evaluator(optimized_react)
print(f"Optimized score: {optimized_score}")

Average Metric: 54.00 / 100 (54.0%): 100%|██████████| 100/100 [00:04<00:00, 21.86it/s]

2025/10/29 10:27:41 INFO dspy.evaluate.evaluate: Average Metric: 54 / 100 (54.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...",I found that Bud Ekins was Steve McQueen's stunt double in the fil...,The Great Escape,✔️ [True]
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,{'thought_0': 'I need to find out which family had their own reali...,"The Kardashian family, associated with Robert Kardashian, has thei...",Robert Kardashian,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in ""Shadows in Paradi...","In my search for the cast of ""Shadows in Paradise,"" I found that t...",Sofya Skya,✔️ [True]
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,"{'thought_0': ""I need to find out who appointed Amashsai and the m...","Amashsai was appointed by Nehemiah, and the name Amasai, which is ...","""Burdensome""",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'I need to find out what additional requirements are...,The search results indicate that Austrian citizens have visa-free ...,"A valid Austrian passport, and potentially a visa or health docume...",
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the release date of the first al...,I found that the American actress and singer-songwriter Katey Saga...,"April 19, 1994",✔️ [True]
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'I need to identify the animated creatures that were...,The question pertains to animated creatures that are the title cha...,"Fairies (specifically Puck, Titania, and Oberon)",
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,"{'thought_0': ""I need to determine if both Dorothy Arzner and Rich...","I found that Dorothy Arzner was an American film director, and Ric...","No, neither Dorothy Arzner nor Richard Wallace were French film di...",


🏃 View run victorious-ox-919 at: https://s172-29-112-60p8080.lab-aws-production.deeplearning.ai//#/experiments/714844631808117306/runs/7817091b4e9e49c0981f05d46d33dc04
🧪 View experiment at: https://s172-29-112-60p8080.lab-aws-production.deeplearning.ai//#/experiments/714844631808117306
Optimized score: 54.0


[Trace(request_id=eab6d7c54eae4e259345bd4db36dd3c0), Trace(request_id=83ce2cafd3184207986a1ee6f9af4342), Trace(request_id=d20f775b85cf4059b04fe111acfba7b4), Trace(request_id=3cb377e2a8644532a4167ba4e1ace6ed), Trace(request_id=62c497298c9f4de9b3bab478395e0527), Trace(request_id=b3f37d1f9c674ad6b99e71d6dca5c9ca), Trace(request_id=5a8b02967f7d4c2fbd5b9b909e247c1f), Trace(request_id=3c36cdd18b5d42fe8f0d2d02b71e0041), Trace(request_id=c01a240fa5da4a2295999b1e164427b6), Trace(request_id=103a14bd6f4440c7977919dd835aed15)]